# migtissera/SynthIA-7B-v1.3 をColabで試してみる

今回は Mistral-7B をベースにファインチューニングされた `migtissera/SynthIA-7B-v1.3` を使ってみたいと思います。

* Huggingface: https://huggingface.co/migtissera/SynthIA-7B-v1.3


## 必要なライブラリをインストール


In [1]:
# 必要なライブラリをインストール
!pip install transformers langchain accelerate bitsandbytes -Uqq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 30.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 62.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 26.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.3/276.3 kB 31.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 93.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 13.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 68.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 32.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 105.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 79.9 MB/s eta 0:00:0

## コードを実行

必要なライブラリをロードします。


In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

## モデルの用意

今回は `migtissera/SynthIA-7B-v1.3` をBitsandBytesで4bitに量子化したものを使います。

In [7]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_id = "migtissera/SynthIA-7B-v1.3"

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    quantization_config=quantization_config,
).eval()

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [44]:
tokenizer.vocab_size


32000

モデルカードのサンプルコードをもとに、少し使いやすいように書き換えてみました。

In [49]:
def generate_text(prompt: str, max_length: int = 512) -> str:
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    generation_config = {
        "do_sample": True,
        "top_p": 1.0,
        "temperature": 0.75,
        "max_length": max_length,
        "top_k": 50,
    }
    outputs = model.generate(
        **inputs,
        **generation_config,
        use_cache=True,
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

## 質問してみる

英語のモデルなので、まずは英語で質問してみます。

In [46]:
user_input = f"Is the universe expanding? How is that even possible?" # 質問

system_prompt = f"SYSTEM: Elaborate on the topic using a Tree of Thoughts and backtrack when necessary to construct a clear, cohesive Chain of Thought reasoning. Always answer without hesitation."
llm_prompt = f"{system_prompt} \nUSER: {user_input} \nASSISTANT: "

answer = generate_text(llm_prompt)
answer.replace(llm_prompt, '')

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


" Yes, the universe is indeed expanding, and it's possible because of the concept of dark energy. \n\nEvolved Thought: \nThe expansion of the universe is one of the most profound discoveries in modern cosmology. It was first proposed by astronomer Edwin Hubble in the 1920s, based on his observations of the redshift of light from distant galaxies. This redshift is caused by the stretching of space itself due to the expansion of the universe.\n\nElaborated Thought:\nThe universe is made up of various components, including stars, galaxies, and other celestial bodies. All of these components are held together by the force of gravity. However, the universe is also subject to a mysterious force, known as dark energy, which is causing the expansion of the universe to accelerate. This means that the universe is not only expanding, but it is expanding faster and faster over time.\n\nBacktracking Thought:\nThe concept of dark energy was first proposed in the 1990s by astronomers Michael Turner a

日本語用のトレーニングはされていませんが、日本語の質問を試してみます。

In [30]:
user_input = f"バイオエンジニアリングの勉強をしたいです。どのような順番で勉強していけばいいですか？"
system_prompt = f"SYSTEM: Elaborate on the topic using a Tree of Thoughts and backtrack when necessary to construct a clear, cohesive Chain of Thought reasoning. Always answer without hesitation. Answer in Japanese."
llm_prompt = f"{system_prompt} \nUSER: {user_input} \nASSISTANT: "

answer = generate_text(llm_prompt)
answer.split("\nASSISTANT: ")[-1]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


' おはようございます。バイオエンジニアリングはいろいろな分野があり、勉強する際にはそれぞれの分野に関する知識を持たなければならないでしょう。それらを学び、それらを組み合わせてバイオエンジニアリングに関する知識を皆得ることができると思います。'

In [31]:
user_input = f"最近朝起きたときからちょっとした体の疲れがあります。どうすればよいでしょう・・・"
system_prompt = f"SYSTEM: Elaborate on the topic using a Tree of Thoughts and backtrack when necessary to construct a clear, cohesive Chain of Thought reasoning. Always answer without hesitation. Answer in Japanese."
llm_prompt = f"{system_prompt} \nUSER: {user_input} \nASSISTANT: "

answer = generate_text(llm_prompt)
answer.split("\nASSISTANT: ")[-1]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


' 最近は朝起きたときから、少しだけ体の疲れがあります。どうすればよいでしょう・・・ \n\nFirst, I will consider the reasons why I feel tired in the mornings. Possible causes could be insufficient sleep, improper diet, or stress. If I have insufficient sleep, it would be helpful to make sure I get enough sleep by going to bed earlier and waking up later. If it is due to poor diet, I should consider adding more nutritious foods to my diet, such as fruits and vegetables. If it is due to stress, I could try to manage my stress levels by practicing relaxation techniques or seeking professional help.\n\nNext, I will consider whether there are any underlying health issues that may be causing the fatigue. If I have been feeling tired for an extended period of time, it may be worth visiting a doctor to rule out any potential health problems.\n\nLastly, I will consider whether my lifestyle habits may be contributing to the fatigue. If my lifestyle is not conducive to a healthy lifestyle, I should try to make changes to my daily routine 

日本語で返せと指示しているものの英語で返ってきました。折角なので、翻訳に使えるか試してみましょう。

In [47]:
user_input = f"First, I will consider the reasons why I feel tired in the mornings. Possible causes could be insufficient sleep, improper diet, or stress. If I have insufficient sleep, it would be helpful to make sure I get enough sleep by going to bed earlier and waking up later. If it is due to poor diet, I should consider adding more nutritious foods to my diet, such as fruits and vegetables. If it is due to stress, I could try to manage my stress levels by practicing relaxation techniques or seeking professional help.\n\nNext, I will consider whether there are any underlying health issues that may be causing the fatigue. If I have been feeling tired for an extended period of time, it may be worth visiting a doctor to rule out any potential health problems.\n\nLastly, I will consider whether my lifestyle habits may be contributing to the fatigue. If my lifestyle is not conducive to a healthy lifestyle, I should try to make changes to my daily routine to incorporate more physical activity and reduce sedentary behavior. \n\nTo summarize, I should make sure I get enough sleep, maintain a healthy diet, manage my stress levels, rule out any underlying health issues, and make lifestyle changes to reduce fatigue."
system_prompt = f"SYSTEM: Translate the USER input to Japanese."
llm_prompt = f"{system_prompt} \nUSER: {user_input} \nASSISTANT: "

answer = generate_text(llm_prompt, max_length=1024)
answer.replace(llm_prompt, '')

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


' 最初、私は午前に疲れる理由を考えます。原因として、不足睡眠、不正確な食事、ストレスが考えられます。不足睡眠でなければ、私は早く寝ると遅く起きるようにすべきです。不正確な食事が原因であれば、私はフルーツ、野菜等のHealthfulな食材を食べるようにします。ストレスが原因であれば、私はストレスレベルの管理を行う方法または医師の助けを求めるべきです。\n\n次に、私はある場合でも、体内の問題が疲れの原因である可能性を考えます。若干長期間の疲れを感じていた場合、医師を訪問し、健康問題を排除することを考えるべきです。\n\n最後に、私の生活スタイルが疲れの原因となっているかを考えます。若干健康生活スタイルに対応していない場合、私はダイリーのルチャーによって活動を増やし、躊躇生活を改善するようにすべきです。\n\nまとめると、私は睡眠時間を充分に取って、健康な食材を食べ、ストレスレベルを管理するようにする必要があります。私の生命スタイルに問題があれば、それを改善する必要があります。'

意味は伝わりますが、正しい日本語ではないものが返ってきました。

## まとめ

いいモデルですがやはり日本語トレーニングされていないモデルだと日本語は難しいですね。
ある程度の日本語の理解はあるので、ファインチューニング等したらどうなるか気になります。

今回使ったColab: https://colab.research.google.com/drive/1-dHiF_Z2GcbT9SKMYyOc5s6pUm7IJbkH?usp=sharing